In [1]:
import pandas as pd
import dbconfig as dbConf

In [2]:
def sql_direct(sql_list):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = sql_list

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [4]:
# 전체 내용
sql = """
# 점이력 조회 
WITH DEPT_MNG (DEPT_NO, EMP_NO, CHG_DT) AS  /* 부서 관리자 데이터 */
(
	SELECT 'DEPT001', 'E0001', '2020-01-02'
	UNION	ALL
	SELECT 'DEPT001', 'E0015', '2020-01-13'
	UNION	ALL
	SELECT 'DEPT001', 'E0009', '2020-01-25'
)
SELECT
		A.DEPT_NO		/* 부서번호 */
	  , A.EMP_NO		/* 관리자 사원번호 */
	  , A.CHG_DT ST_DT	/* 변경일자(시작일자) */
	  ,	CASE
			WHEN B.CHG_DT IS NULL
				THEN '(현재)'
				ELSE B.CHG_DT
		END ED_DT		/* 종료일자 */
FROM	(
			SELECT
					DEPT_NO
				  , EMP_NO
				  , CHG_DT
				  , ROW_NUMBER() OVER(PARTITION BY DEPT_NO ORDER BY CHG_DT) RN
			FROM	DEPT_MNG
			WHERE	1 = 1
			AND		DEPT_NO = 'DEPT001'
		)	A
LEFT	JOIN (
				SELECT
						DEPT_NO
					  , EMP_NO
					  , CHG_DT
					  , ROW_NUMBER() OVER(PARTITION BY DEPT_NO ORDER BY CHG_DT) RN
				FROM	DEPT_MNG
				WHERE	1 = 1
				AND		DEPT_NO = 'DEPT001'
			)	B
ON		A.DEPT_NO = B.DEPT_NO
AND		A.RN = B.RN - 1
WHERE	1 = 1
ORDER	BY A.DEPT_NO, ST_DT DESC
;
       
"""

sql_all_data = sql_direct(sql)

sql_all_data

,DEPT_NO,EMP_NO,ST_DT,ED_DT
0,DEPT001,E0009,2020-01-25,(현재)
1,DEPT001,E0015,2020-01-13,2020-01-25
2,DEPT001,E0001,2020-01-02,2020-01-13


In [5]:
# 전체 내용
sql = """
# 선분이력 조회
WITH DEPT_MNG (DEPT_NO, EMP_NO, ST_DT, ED_DT) AS  /* 부서 관리자 데이터 */
(
	SELECT 'DEPT001', 'E0001', '2020-01-02', '2020-01-13'
	UNION	ALL
	SELECT 'DEPT001', 'E0015', '2020-01-13', '2020-01-25'
	UNION	ALL
	SELECT 'DEPT001', 'E0009', '2020-01-25', NULL
)
SELECT
		DEPT_NO		/* 부서번호 */
	  , EMP_NO		/* 관리자 사원번호 */
	  , ST_DT		/* 시작일자 */
	  ,	CASE
			WHEN ED_DT IS NULL
				THEN '(현재)'
				ELSE ED_DT
		END ED_DT	/* 종료일자 */
FROM	DEPT_MNG
WHERE	1 = 1
AND		DEPT_NO = 'DEPT001'
ORDER	BY	ST_DT DESC
;
"""

sql_all_data = sql_direct(sql)

sql_all_data

,DEPT_NO,EMP_NO,ST_DT,ED_DT
0,DEPT001,E0009,2020-01-25,(현재)
1,DEPT001,E0015,2020-01-13,2020-01-25
2,DEPT001,E0001,2020-01-02,2020-01-13
